In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

C:\Anaconda3\envs\py39tf2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(np.exp(0))
print(np.exp(1))
print(np.exp(2))
print(np.exp(3))

1.0
2.718281828459045
7.38905609893065
20.085536923187668


In [4]:
np.exp(1) / (np.exp(1) + np.exp(2) + np.exp(3)) 

0.09003057317038046

In [5]:
z = torch.FloatTensor([1,2,3])
print(z)

tensor([1., 2., 3.])


In [13]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [14]:
# 전체 값은 1
hypothesis.sum()

tensor(1.)

In [5]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.1652, 0.1313, 0.1780, 0.3433, 0.1823],
        [0.0968, 0.2355, 0.2218, 0.2116, 0.2342],
        [0.2384, 0.2905, 0.1787, 0.1422, 0.1501]], grad_fn=<SoftmaxBackward0>)


In [6]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([2, 3, 4])


In [7]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [8]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.7252, grad_fn=<MeanBackward0>)


In [9]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.8009, -2.0305, -1.7261, -1.0693, -1.7019],
        [-2.3347, -1.4459, -1.5058, -1.5533, -1.4515],
        [-1.4336, -1.2361, -1.7219, -1.9509, -1.8961]], grad_fn=<LogBackward0>)

In [10]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.8009, -2.0305, -1.7261, -1.0693, -1.7019],
        [-2.3347, -1.4459, -1.5058, -1.5533, -1.4515],
        [-1.4336, -1.2361, -1.7219, -1.9509, -1.8961]],
       grad_fn=<LogSoftmaxBackward0>)

In [11]:
F.cross_entropy(z, y)

tensor(1.7252, grad_fn=<NllLossBackward0>)

In [47]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [50]:
# 모델 초기화
w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer= optim.SGD([w, b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(w) + b, dim =1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


In [38]:
# 모델 초기화
w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer= optim.SGD((w, b), lr = 0.1)

In [39]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산 (2)
    z = x_train.matmul(w) + b
    cost = F.cross_entropy(z, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [40]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
        
    def forward(self, x):
        return self.linear(x)

In [43]:
model = SoftmaxClassifierModel()

In [44]:
# optimizer 설정
optimizer= optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 설정
    prediction = model(x_train)
    
    # cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.644511
Epoch  100/1000 Cost: 0.710236
Epoch  200/1000 Cost: 0.627504
Epoch  300/1000 Cost: 0.569449
Epoch  400/1000 Cost: 0.518588
Epoch  500/1000 Cost: 0.470717
Epoch  600/1000 Cost: 0.424215
Epoch  700/1000 Cost: 0.378304
Epoch  800/1000 Cost: 0.332679
Epoch  900/1000 Cost: 0.287945
Epoch 1000/1000 Cost: 0.250265
